This notebook mainly cover image classification, and simplified input using Kaggle's data (no need to download, so it is fast and a lot in use).

Linke to [CheatSheet 2 (Recurrent Networks, Time Series and Natural Language Processing)](https://www.kaggle.com/yyyaaan/my-cheatsheet-tensorflow-keras-2)

> Compiled with GPU enabled. A global callback is enabled so that model training stops when accuracy exceeds 97%.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.preprocessing import image
from keras import layers

opt_verbose = 1 if 'runtime' in get_ipython().config.IPKernelApp.connection_file else 2

# Use of callbacks to stop training once it is good enough

Regardless of `epochs` parameter, model training can be stopped if its accuracy or loss is good enough.

This section use in-memory data [Fashio MNIST from Kaggle](https://www.kaggle.com/zalando-research/fashionmnist), and it is also bundled in `tf.keras.datasets.fashion_mnist`. [Data Source on Github](https://github.com/zalandoresearch/fashion-mnist). The labels include 10 classes.

In [ ]:
fashion_mnist_train = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
fashion_mnist_test = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')
# Pop will also remove column from original dataset
fashion_mnist_train_label = fashion_mnist_train.pop('label')
fashion_mnist_test_label = fashion_mnist_test.pop('label')

fashion_mnist_train.shape, fashion_mnist_train_label.shape, fashion_mnist_test.shape, fashion_mnist_test_label.shape

In [ ]:
# Reshape and Standardize to 0-1
fashion_mnist_train1=np.array(fashion_mnist_train).reshape(60000, 28, 28, 1)
fashion_mnist_test1=np.array(fashion_mnist_test).reshape(10000, 28, 28, 1)
fashion_mnist_train1, fashion_mnist_test1 = fashion_mnist_train1/255., fashion_mnist_test1/255.

## Define myCallback Class

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.97):
      print("--> Callback: Reached 97% accuracy. Cheers and relax! <--")
      self.model.stop_training = True

callbacks = myCallback()
max_epochs = 15

## Run model with callback and validation split

In [ ]:
model1 = Sequential([
  layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  layers.MaxPooling2D(2, 2),
  layers.Conv2D(64, (3,3), activation='relu'),
  layers.MaxPooling2D(2,2),
  layers.Flatten(),
  layers.Dense(512, activation=tf.nn.relu),
  layers.Dense(10, activation=tf.nn.softmax),    
])

model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

hisfit=model1.fit(
    fashion_mnist_train1, 
    fashion_mnist_train_label, 
    epochs=max_epochs, 
    validation_split=0.1,
    callbacks=[callbacks], 
    verbose=opt_verbose,
)


hisevl=model1.evaluate(fashion_mnist_test1, fashion_mnist_test_label, verbose=opt_verbose)

# Preprocessing, flow and augmentation

`ImageDataGenerator` does not load data into memory. With `flow_from_directory()`, it is possible to resize, rotation, flip, shift and rescale. Augmentation is done in-memory.

Data Source: [Horses or Humans on Kaggle](https://www.kaggle.com/sanikamal/horses-or-humans-dataset) and orginally available [here](http://www.laurencemoroney.com/horses-or-humans-dataset/). Label is binary.

References: [Colab1](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%201%20-%20Part%208%20-%20Lesson%202%20-%20Notebook.ipynb) | [Colab2](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%204%20-%20Lesson%204%20-%20Notebook.ipynb)

## Define a flow from directory

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',    
)

validation_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
    '../input/horses-or-humans-dataset/horse-or-human/train/',
    target_size=(300, 300), 
    batch_size=128,
    class_mode='binary',
)

validation_generator = validation_datagen.flow_from_directory(
    '../input/horses-or-humans-dataset/horse-or-human/validation/',
    target_size=(300, 300), 
    batch_size=128,
    class_mode='binary',
)

## A vision model with RMSprop and validation

Learning rate tuning is discussed in [Cheatsheet 2](https://www.kaggle.com/yyyaaan/my-cheatsheet-tensorflow-keras-2)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model2 =Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

model2.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

In [ ]:
hisfit=model2.fit(
    train_generator, 
    steps_per_epoch=8,
    epochs=max_epochs,
    validation_data=validation_generator,
    validation_steps=8,
    callbacks=[callbacks],
    verbose=opt_verbose,
)

# Transferred learning

Keyword: Multi-class, pre-trained, dropout, split, 

Reference: [Colab1](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%206%20-%20Lesson%203%20-%20Notebook.ipynb#scrollTo=Blhq2MAUeyGA) | [Colab2](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%208%20-%20Lesson%202%20-%20Notebook%20(RockPaperScissors).ipynb) | [Data Source](http://www.laurencemoroney.com/rock-paper-scissors-dataset/)

## Load pre-trained, set non-trainable and last layer

In [ ]:
!wget --no-check-certificate -q\
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O ./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (300, 300, 3), include_top = False, weights = None)
pre_trained_model.load_weights('./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
for layer in pre_trained_model.layers:
  layer.trainable = False

# specify which layer to be the last layer
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

## Continue Model with own layers

In [ ]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x) 
x = layers.Dense(3, activation='softmax')(x)

model3 = tf.keras.Model(pre_trained_model.input, x) 

# LOSS is different with binary
model3.compile(optimizer=RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])


## Split use data generator

For folders not organized in training and validation, it is possible to split. First use `validation_split=0.2` and then in flow use `subset="training/validation"`. Remeber never augment picture in validation.

In [ ]:
# the validation_split here, and subset below
better_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1/255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',    
)

# excluding some files/directories using classes
train_generator = better_datagen.flow_from_directory(
    '../input/rockpaperscissors',
    target_size=(300, 300), 
    batch_size=64,
    classes=('paper', 'rock', 'scissors'),
    class_mode='categorical',
    subset="training",
)

validation_generator = better_datagen.flow_from_directory(
    '../input/rockpaperscissors',
    target_size=(300, 300), 
    batch_size=64,
    classes=('paper', 'rock', 'scissors'),
    class_mode='categorical',
    subset='validation',
)

In [ ]:
hisfit=model3.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 64,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 64,
    epochs = 10,
    callbacks=[callbacks], 
    verbose=opt_verbose,
)

# Utilities for quick actions

## Plot loss and accuracy

In [ ]:
def plot_history(history):
    import matplotlib.image  as mpimg
    import matplotlib.pyplot as plt

    epochs=range(len(history['accuracy'])) # Get number of epochs

    plt.plot(epochs, history['accuracy'], 'r', "Training Accuracy")
    plt.plot(epochs, history['val_accuracy'], 'b', "Validation Accuracy")
    plt.title('Training and validation accuracy')
    
    plt.figure()
    plt.plot(epochs, history['loss'], 'r', "Training Loss")
    plt.plot(epochs, history['val_loss'], 'b', "Validation Loss")
    plt.title('Training and validation loss')

In [ ]:
%matplotlib inline
plot_history(hisfit.history)

## Serving a vision model

It is a pseudo example! change URL to get your own.

In [ ]:
# download here (always with name tmppic)
!wget -q -O ../tmppic https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Rock-paper-scissors_%28scissors%29.png/900px-Rock-paper-scissors_%28scissors%29.png

path="../tmppic"
size=300

img = image.load_img(path, target_size=(size, size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model3.predict(images, batch_size=10)


plt.imshow(img)
print(f"Predict: {np.argmax(classes, axis=1)}. Probabiilty: {classes}")